In [7]:
import numpy as np

In [11]:
class HiddenMarkov:
    def forward(self, Q, V, A, B, O, PI):  # 使用前向算法
        N = len(Q)  # 状态序列的大小
        M = len(O)  # 观测序列的大小
        alphas = np.zeros((N, M))  # alpha值
        T = M  # 有几个时刻，有几个观测序列，就有几个时刻
        for t in range(T):  # 遍历每一时刻，算出alpha值
            indexOfO = V.index(O[t])  # 找出序列对应的索引
            for i in range(N):
                if t == 0:  # 计算初值
                    alphas[i][t] = PI[t][i] * B[i][indexOfO] 
                else:
                    alphas[i][t] = np.dot([alpha[t - 1] for alpha in alphas], [a[i] for a in A]) * B[i][
                        indexOfO]  
        P = np.sum([alpha[M - 1] for alpha in alphas]) 
        print("观测序列出现的概率为",P)
        return(alphas)
       
    def backward(self, Q, V, A, B, O, PI):  # 后向算法
        N = len(Q)  # 状态序列的大小
        M = len(O)  # 观测序列的大小
        betas = np.ones((N, M))  # beta                                    
        for t in range(M - 2, -1, -1):
            indexOfO = V.index(O[t + 1])  # 找出序列对应的索引
            for i in range(N):
                betas[i][t] = np.dot(np.multiply(A[i], [b[indexOfO] for b in B]), [beta[t + 1] for beta in betas])
        indexOfO = V.index(O[0])
        P = np.dot(np.multiply(PI, [b[indexOfO] for b in B]), [beta[0] for beta in betas])
        print("观测序列出现的概率为",P)
        return(betas)

    def viterbi(self, Q, V, A, B, O, PI):   #维特比算法
        N = len(Q)  # 状态序列的大小
        M = len(O)  # 观测序列的大小
        deltas = np.zeros((N, M))
        psis = np.zeros((N, M))
        I = np.zeros((1, M))
        #递推
        for t in range(M):
            indexOfO = V.index(O[t])  # 找出序列对应的索引
            for i in range(N):
                if t == 0:
                    deltas[i][t] = PI[0][i] * B[i][indexOfO]
                    psis[i][t] = 0
                else:
                    deltas[i][t] = np.max(np.multiply([delta[t-1] for delta in deltas], [a[i] for a in A])) * B[i][indexOfO]
                    psis[i][t] = np.argmax(np.multiply([delta[t-1] for delta in deltas], [a[i] for a in A]))
        print(deltas)
        print(psis)
        #最优路径回溯
        I[0][M-1] = np.argmax([delta[M-1] for delta in deltas])
        for t in range(M-2, -1, -1):
            I[0][t] = psis[int(I[0][t+1])][t+1]
        print(I)
        
    
    def baum_welch_train(self, O, criterion=0.05, A, B, PI):
        n_states = A.shape[0]
        # 观察序列的长度T
        n_samples = len(O)

        done = False
        while not done:
        alpha = self.forward(O)
        beta = self.backward(O)
        
        #计算delta
        xi = np.zeros((n_states,n_states,n_samples-1))
        for t in range(n_samples-1):
            denom = np.dot(np.dot(alpha[:,t].T, A) * B[:,O[t+1]].T, beta[:,t+1])
            for i in range(n_states):
                numer = alpha[i,t] * A[i,:] * B[:,O[t+1]].T * beta[:,t+1].T
                xi[i,:,t] = numer / denom
  
        #计算gamma_t(i) 就是对j进行求和
        gamma = np.sum(xi, axis=1) 
        # need final gamma elements for new B
        prod = (alpha[:, n_samples - 1] * beta[:, n_samples - 1]).reshape((-1, 1))
        # 合并T时刻的节点
        gamma = np.hstack((gamma, prod / np.sum(prod)))
            # 列向量
            newpi = gamma[:, 0]
            newA = np.sum(xi, 2) / np.sum(gamma[:, :-1], axis=1).reshape((-1, 1))
            newB = np.copy(self.B)

            # 观测状态数
            num_levels = self.B.shape[1]
            sumgamma = np.sum(gamma, axis=1)
            for lev in range(num_levels):
                mask = observations == lev
                newB[:, lev] = np.sum(gamma[:, mask], axis=1) / sumgamma

            if np.max(abs(self.pi - newpi)) < criterion and \
                            np.max(abs(self.A - newA)) < criterion and \
                            np.max(abs(self.B - newB)) < criterion:
                done = 1
            A[:], self.B[:], self.pi[:] = newA, newB, newpi


In [14]:
#例10.3
Q = [1, 2, 3]
V = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
O = ['红', '白', '红']    
PI = [[0.2, 0.4, 0.4]]

In [15]:
HMM = HiddenMarkov()
HMM.viterbi(Q, V, A, B, O, PI)

[[ 0.1      0.028    0.00756]
 [ 0.16     0.0504   0.01008]
 [ 0.28     0.042    0.0147 ]]
[[ 0.  2.  1.]
 [ 0.  2.  1.]
 [ 0.  2.  2.]]
[[ 2.  2.  2.]]


In [2]:
#习题10.2
Q = [1, 2, 3]
V = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
O = ['红', '白', '红', '红', '白', '红', '白', '白']
PI = [[0.2, 0.3, 0.5]]

In [10]:
HMM.forward(Q, V, A, B, O, PI)
HMM.backward(Q, V, A, B, O, PI)

观测序列出现的概率为 0.00385197357949
观测序列出现的概率为 [ 0.00385197]
